In [1]:
#   |'''''''''''''╔╬╬╬╬╬╬╬╬   _____  _____      _____  _____      ___   __
#   |            ╔╬╬╬╬╬╬╬╬╬  |\   _ \  _  \    |\   _ \  _  \    |\  \|\  \
#   | ░░         ╬╬╬╬╬╬╬╬╬╬  \ \  \\__\ \  \   \ \  \\__\ \  \   \ \  \/  /|_
#    ░░░░        ╬╬╬╬╬╬╬╬╬╬   \ \  \|__| \  \   \ \  \|__| \  \   \ \   ___  \
#   ░░░░░╦╬╦    ╔╬╬╬╬╬╬╬╬╬╬    \ \  \   \ \  \   \ \  \   \ \  \   \ \  \ \   \
#  ░░░░░╬╬╬╬ ▓▓└╬╬╬╬╬╬╬╬╬╬╬     \ \__\   \ \__\   \ \__\   \ \__\   \ \__\ \___\
# ░░░░░╔╬╬╬ ▓▓▓  ╓╬╬╬╬╬╬╬╬╬      \|__|    \|__|    \|__|    \|__|    \|__| \|__|
# ░░░░░╠╬╬╬ ▓▓▓  └╬╬╬╬╬╬╬╬╬
#  ░░░░└╬╬╬╬ ▓▓   ╬╬╬╬╬╬╬╬╬  Lehrstuhl für Mensch-Maschine-Kommunikation
#  ░░░░░╙╬╬╬╩            ╬╬  Technische Universität München
#   ░░░░░░╚ '''''''''''''''  Author: Tobias Watzel
#    ░░░                     Copyright 2020
#

%matplotlib widget
import cv2
import glob
import re
import io
import ipywidgets as widgets
import numpy as np
from IPython.display import clear_output, Image, display
import PIL.Image
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib as mpl
from matplotlib import cm

# define figure
fig = None

## Versuchsbeschreibung
Wählen Sie ein Bild und vergleichen Sie die Histogramme der technischen und der wahrnehmungsgerechten Grauwertkonvertierung.

In [2]:
if fig:
    plt.close(fig)

fig = plt.figure(figsize = (9, 6))
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)
# create subaxes
left, bottom, width, height = ax1.get_position().bounds
ax1_subaxes = fig.add_axes([left, bottom - height * 0.05 - 0.01, width, height * 0.05])
left, bottom, width, height = ax2.get_position().bounds
ax2_subaxes = fig.add_axes([left, bottom - height * 0.05 - 0.01, width, height * 0.05])
norm = mpl.colors.Normalize(vmin = 0, vmax = 255)
cb1 = mpl.colorbar.ColorbarBase(ax1_subaxes, norm = norm, orientation = 'horizontal', cmap = cm.Greys_r)
cb1 = mpl.colorbar.ColorbarBase(ax2_subaxes, norm = norm, orientation = 'horizontal', cmap = cm.Greys_r)
ax1.get_xaxis().set_visible(False)
ax2.get_xaxis().set_visible(False)
#hist2 = ax2.hist(np.zeros(1))

# create dict with name of image and its path (FIX? hacked...)
dict_images = {entry.replace('/',' ').replace('.',' ').split()[1]:entry for entry in sorted(glob.glob("images/*"))}

def convert_image(a, fmt='png'):
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    return f.getvalue()

def rgb_to_gray_colorimetric(image_in):
    # check, if RGB
    assert image_in.shape[2] == 3
    return 0.3 * image_in[:, :, 0] + 0.59 * image_in[:, :, 1] + 0.11 * image_in[:, :, 2]

def rgb_to_gray_average(image_in):
    # check, if RGB
    assert image_in.shape[2] == 3
    return np.mean(image_in, axis=2)

def display_images(image_name):  
    # read image
    image = cv2.imread(dict_images[image_name])
    
    # image in RGB (just switching dims)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # transform images
    image_gray_average = rgb_to_gray_average(image_rgb)
    image_gray_colorimetric = rgb_to_gray_colorimetric(image_rgb)
    
    # plot histgrams
    ax1.cla()
    ax2.cla()
    ax1.set_title('Technisches Histogram')
    ax2.set_title('Wahrnehmungsgerechtes Histogram')
    ax1.hist(image_gray_average.ravel(), 256, [0, 256])
    ax2.hist(image_gray_colorimetric.ravel(), 256, [0, 256])
    ax1.set_xlim(0, 255)
    ax2.set_xlim(0, 255)

    
def change_val(change):
    if change['type'] == 'change' and change['name'] == 'value':
        display_images(change['new'])

        
select = widgets.Select(
    options=list(dict_images.keys()),
    value='birne',
    description='Dateiliste:')

display(select)
display_images('birne')

select.observe(change_val)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Select(description='Dateiliste:', options=('birne', 'buch', 'fahrzeug', 'klammermaschine', 'mandrill', 'orange…

# Vergleichung
## Der Unterscheid gibt es in der Verteilung der Helligkeitswerte. Meistens Wahrnehmungsgerechte Grauwertkonvertiertung ist eindeutiger als technische Grauwertkonvertierung. In dem wahrnehmnungsgerechten Histogram für rgb_stripes_color, gibt es drei Linie, die rot, grün und blau erkannt werden. Im vergelcieh dazu, in dem technischen Histogram kann man sehen, nur eine Linie. Das heißt, wahrnehmungsgerechten Grauwert  als technisches kann besser und eindeutiger als technisches aufgefallen werden.